In [2]:
import pandas as pd
import numpy as np
import re
import time
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore')

import os
import sys
import urllib.request
import json
client_id = "BWHbbS5kayySrc7QiSV7" 
client_secret = "DB9ssS2KBq" 



In [59]:
### [1] URL 생성관련 함수
def file_to_df():
    df = pd.read_excel('apple_iphone_url.xlsx')
    df['inch'] = df['inch'].astype(str)
    df['germany_inch'] = df['inch'].apply(lambda x: re.sub("\.",",",x))
    df['russia_rom'] = df['rom'].apply(lambda x: re.sub("gb","",x))
    
    return df
    
def url_maker(df):
    df['us_url'] = "https://www.apple.com/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']+"-unlocked"
    df['uk_url'] = "https://www.apple.com/uk/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']
    df['china_url'] = "https://www.apple.com.cn/shop/buy-iphone/"+df['family_model']+"/"+df['china_code']+"/A"
    
    df['japan_url'] = 0
    df['germany_url'] = 0
    df['russia_url'] = 0

    for i in range(len(df)):
        df['japan_url'][i] = "https://www.apple.com/jp/shop/buy-iphone/"+df['family_model'][i]+"/"+df['inch'][i]+\
                    parse.quote(df['japan_code_inch'][i])+"-"+df['rom'][i]+"-"+parse.quote(df['japan_code_color'][i])
        df['germany_url'][i] = "https://www.apple.com/de/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"%22-display-"+\
                               df['rom'][i]+"-"+parse.quote(df['germany_code'][i]) 

        if df['family_model'][i] == "iphone-se":
            df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%BE%D0%B2%D1%8B%D0%B9-%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])
        else: 
             df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+\
                                    "%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                    df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%B0-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])       

    df['india_url'] = "https://www.indiaistore.com/"+df['model']+"/"
    
    return df

### [2]크롤링 관련 코드
def select_no(browser):
    try:
        no_answer = browser.find_element_by_class_name('as-inlinetradeup-selector')
        no_answer.click()
    except: pass
    #print("click No")
          
def get_delivery_date(browser,post_code):
    #1. No 선택했는지 확인    
    browser.implicitly_wait(10)
    time.sleep(2)
    delivery_date_list = browser.find_elements_by_class_name("as-purchaseinfo-dudeinfo-deliverymsg")
    delivery_date_li=[]

    for i in range(len(delivery_date_list)):
        delivery_date_li.append(delivery_date_list[i].text)
        #print(delivery_date_list[i])
        #print(delivery_date_li)

    delivery_date = " / ".join(delivery_date_li)
    print(delivery_date)
    return delivery_date

def get_price_delivery(browser, url, country,i,post_code):
    browser.get(url)
    #time.sleep(3)
    
    select_no(browser)
    delivery_date = get_delivery_date(browser,post_code)
    price = browser.find_element_by_class_name('localnav-price-content').text
    price_delivery = price+"|"+delivery_date
    return price_delivery

#네이버파파고API
def tranlator_papago(text, src_lang, dest_lang):
    data = "source=" + src_lang + "&target=" + dest_lang + "&text=" + text
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        # print(response_body.decode('utf-8'))

        #json 형 변환
        res = json.loads(response_body.decode('utf-8'))
        
    else:
        print("Error Code:" + rescode)
    return res['message']['result']['translatedText']

In [3]:
# <1> 기본 DataFrame을 세팅한다.
df = file_to_df()
df = url_maker(df)
browser = webdriver.Chrome()

In [14]:
url_list =[('usa','us_url','10001'),('uk','uk_url','sw1a2aa'), ('china', 'china_url','100000'), 
           ('japan', 'japan_url',''), ('germany','germany_url','10115'), 
           ('russia','russia_url','101000'), ('india', 'india_url')]

# <2> 나라를 선택하고 진행 - 0번째 url 받아서 우펀코드 설정
con_num = 5 #<--------------------[입력] usa:0, uk:1, china:2, japan:3, germany:4, russia:5, india:6

country = url_list[con_num][0]
res_col = url_list[con_num][0]+"_res"
col_nm = url_list[con_num][1]
post_code = url_list[con_num][2]
url = df[col_nm][0] #---> us_url, uk_url, china_url, japan_url, germany_url, russia_url, india_url 

#브라우저 열기  --> 우편코드 입력해주어야함  <<<---- 
browser.get(url)

In [15]:
# <3> 우편코드를 설정했다면, 이제 크롤링 GoGo!!!
df[res_col] = 0

for i in range(len(df['model'])):
    print(i, end = " ")
    url = df[col_nm][i]
    price_delivery_list = get_price_delivery(browser, url,country,i,post_code)
    df[res_col][i] = price_delivery_list
    
display(df.head(2))

0 Ср 25 Ноя – Бесплатно
1 Ср 25 Ноя – Бесплатно
2 Ср 25 Ноя – Бесплатно
3 Ср 25 Ноя – Бесплатно
4 Ср 25 Ноя – Бесплатно
5 Ср 25 Ноя – Бесплатно
6 Ср 25 Ноя – Бесплатно
7 Ср 25 Ноя – Бесплатно
8 Ср 25 Ноя – Бесплатно
9 Ср 25 Ноя – Бесплатно
10 Ср 25 Ноя – Бесплатно
11 Ср 25 Ноя – Бесплатно
12 Ср 25 Ноя – Бесплатно
13 Ср 25 Ноя – Бесплатно
14 Ср 25 Ноя – Бесплатно
15 Ср 25 Ноя – Бесплатно
16 Ср 25 Ноя – Бесплатно
17 Ср 25 Ноя – Бесплатно
18 Ср 25 Ноя – Бесплатно
19 Ср 25 Ноя – Бесплатно
20 Ср 25 Ноя – Бесплатно
21 Ср 25 Ноя – Бесплатно
22 Ср 25 Ноя – Бесплатно
23 Ср 25 Ноя – Бесплатно
24 Ср 25 Ноя – Бесплатно
25 Ср 25 Ноя – Бесплатно
26 Ср 25 Ноя – Бесплатно
27 Ср 25 Ноя – Бесплатно
28 Ср 25 Ноя – Бесплатно
29 Ср 25 Ноя – Бесплатно
30 9 Дек - 16 Дек – Бесплатно
31 9 Дек - 16 Дек – Бесплатно
32 9 Дек - 16 Дек – Бесплатно
33 9 Дек - 16 Дек – Бесплатно
34 9 Дек - 16 Дек – Бесплатно
35 9 Дек - 16 Дек – Бесплатно
36 9 Дек - 16 Дек – Бесплатно
37 9 Дек - 16 Дек – Бесплатно
38 9 Дек - 16 Дек – 

,No,model,family_model,inch,rom,color,china_code,japan_code_inch,japan_code_color,germany_code,...,japan_url,germany_url,russia_url,india_url,usa_res,uk_res,china_res,japan_res,germany_res,russia_res
0,1,iphone-12,iphone-12,6.1,64gb,white,MGGN3CH,インチディスプレイ,ホワイト,weiß,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно
1,2,iphone-12,iphone-12,6.1,64gb,black,MGGM3CH,インチディスプレイ,ブラック,schwarz,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно
2,3,iphone-12,iphone-12,6.1,64gb,blue,MGGQ3CH,インチディスプレイ,ブラック,blau,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно
3,4,iphone-12,iphone-12,6.1,64gb,green,MGGT3CH,インチディスプレイ,グリーン,grün,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|2020/12/03 - 2020/12/08 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно
4,5,iphone-12,iphone-12,6.1,64gb,red,MGGP3CH,インチディスプレイ,レッド,rot,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,59,iphone-se,iphone-se,4.7,128gb,black,MHH43CH,インチのディスプレイ,ブラック,schwarz,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-se/,$18.70/mo. or $449.00|Tomorrow 12pm - 2pm – Fa...,£18.70/mo. or £449.00**|Wed 25 Nov – Free,RMB 158/月或 RMB 3799|明天 – 免费,"49,800円（税別）|木 2020/11/26 – 送料無料","21,48 € /Monat oder 515,65 €**|Heute 19 Uhr - ...",44 990.00 pyб.|Чт 26 Ноя – Бесплатно
59,60,iphone-se,iphone-se,4.7,128gb,red,MHH63CH,インチのディスプレイ,レッド,rot,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-se/,$18.70/mo. or $449.00|Tomorrow 12pm - 2pm – Fa...,£18.70/mo. or £449.00**|Wed 25 Nov – Free,RMB 158/月或 RMB 3799|明天 – 免费,"49,800円（税別）|木 2020/11/26 – 送料無料","21,48 € /Monat oder 515,65 €**|Heute 19 Uhr - ...",44 990.00 pyб.|Чт 26 Ноя – Бесплатно
60,61,iphone-se,iphone-se,4.7,256gb,white,MHH83CH,インチのディスプレイ,ホワイト,weiß,...,https://www.apple.com/jp/shop/buy-iphone/i

In [150]:
df.to_excel('iphone_New_before_trans_11.23.xlsx')

In [3]:
df = pd.read_excel('iphone_New_before_trans_11.23.xlsx')


In [5]:
df.head(1)

,Unnamed: 0,No,model,family_model,inch,rom,color,china_code,japan_code_inch,japan_code_color,...,japan_url,germany_url,russia_url,india_url,usa_res,uk_res,china_res,japan_res,germany_res,russia_res
0,0,1,iphone-12,iphone-12,6.1,64gb,white,MGGN3CH,インチディスプレイ,ホワイト,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно


In [6]:
# <1> result 결과만 가져와서 새로운 DataFrame 생성
df_b = df[['usa_res', 'uk_res','china_res','japan_res','germany_res','russia_res']]

# <2> "|"를 기준으로 가격과 Delivery Date 구분
df_b['us_price'] = df_b['usa_res'].apply(lambda x : x.split(" or ")[1].split("|")[0])
df_b['us_delivery'] = df_b['usa_res'].apply(lambda x : x.split(" or ")[1].split("|")[1])

df_b['uk_price'] = df_b['uk_res'].apply(lambda x : x.split(" or ")[1].replace("**","").split("|")[0])
df_b['uk_delivery'] = df_b['uk_res'].apply(lambda x : x.split(" or ")[1].replace("**","").split("|")[1])

df_b['ch_price'] = df_b['china_res'].apply(lambda x : x.split("月或 ")[1].split("|")[0])
df_b['ch_delivery'] = df_b['china_res'].apply(lambda x : x.split("月或 ")[1].split("|")[1])

df_b['ja_price'] = df_b['japan_res'].apply(lambda x : x.replace("（税別）","").split("|")[0])
df_b['ja_delivery'] = df_b['japan_res'].apply(lambda x : x.replace("（税別）","").split("|")[1])

df_b['de_price'] = df_b['germany_res'].apply(lambda x : x.split(" oder ")[1].replace("**","").split("|")[0])
df_b['de_delivery'] = df_b['germany_res'].apply(lambda x : x.split(" oder ")[1].replace("**","").split("|")[1])

df_b['ru_price'] = df_b['russia_res'].apply(lambda x : x.split("|")[0])
df_b['ru_delivery'] = df_b['russia_res'].apply(lambda x : x.split("|")[1])

In [17]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   usa_res      63 non-null     object
 1   uk_res       63 non-null     object
 2   china_res    63 non-null     object
 3   japan_res    63 non-null     object
 4   germany_res  63 non-null     object
 5   russia_res   63 non-null     object
 6   us_price     63 non-null     object
 7   us_delivery  63 non-null     object
 8   uk_price     63 non-null     object
 9   uk_delivery  63 non-null     object
 10  ch_price     63 non-null     object
 11  ch_delivery  63 non-null     object
 12  ja_price     63 non-null     object
 13  ja_delivery  63 non-null     object
 14  de_price     63 non-null     object
 15  de_delivery  63 non-null     object
 16  ru_price     63 non-null     object
 17  ru_delivery  63 non-null     object
 18  ch_del_tr    63 non-null     int64 
 19  ja_del_tr    63 non-null     in

In [40]:
from googletrans import Translator

RE_TKK = re.compile(r'TKK=eval\(\'\(\(function\(\)\{(.+?)\}\)\(\)\)\'\);',
                    re.DOTALL)
RE_RAWTKK = re.compile(r'TKK=\'([^\']*)\';',re.DOTALL)

def __init__(self, tkk='0', session=None, host='translate.google.com'):
    self.session = session or requests.Session()
    self.tkk = tkk
    self.host = host if 'http' in host else 'https://' + host

def _update(self):
    """update tkk
    """
    # we don't need to update the base TKK value when it is still valid
    now = math.floor(int(time.time() * 1000) / 3600000.0)
    if self.tkk and int(self.tkk.split('.')[0]) == now:
        return

    r = self.session.get(self.host)

    rawtkk = self.RE_RAWTKK.search(r.text)
    if rawtkk:
        self.tkk = rawtkk.group(1)
        return
    


#번역 후 넣을 빈 컬럼 만들기
df_b[['ch_del_tr', 'ja_del_tr' , 'de_del_tr', 'ru_del_tr']]=0

for i in range(len(df_b)):
    translator = Translator()
    
    chinese_word = df_b['ch_delivery'][i]
    chinese_translated = translator.translate(chinese_word, src='zh-tw', dest='en').text
    df_b['ch_del_tr'][i] = chinese_translated
    
    japanese_word = df_b['ja_delivery'][i]
    japanese_translated = translator.translate(japanese_word, src='ja', dest='en').text
    df_b['ja_del_tr'][i] = japanese_translated 
    
    germany_word = df_b['de_delivery'][i]
    germany_translated = translator.translate(germany_word, src='de', dest='en').text
    df_b['de_del_tr'][i] = germany_translated
    
    russia_word = df_b['ru_delivery'][i]
    russia_translated = translator.translate(russia_word, src='ru', dest='en').text
    df_b['ru_del_tr'][i] = russia_translated
    
    print(chinese_translated,'\n', japanese_translated,'\n', germany_translated,'\n', russia_translated)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [134]:
df_b

,usa_res,uk_res,china_res,japan_res,germany_res,russia_res,us_price,us_delivery,uk_price,uk_delivery,...,ja_price,ja_delivery,de_price,de_delivery,ru_price,ru_delivery,ch_del_tr,ja_del_tr,de_del_tr,ru_del_tr
0,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно,$829.00,"Tomorrow 12pm - 2pm – Fastest / Tue, Nov 24 – ...",£799.00,Wed 25 Nov – Free,...,"85,800円",水 2020/11/25 – 送料無料,"876,30 €",Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,79 990.00 pyб.,Ср 25 Ноя – Бесплатно,星期三 2020/11/25 – 免费,水 2020/11/25 – 送料無料,Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,Ср 25 Ноя – Бесплатно
1,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно,$829.00,"Tomorrow 12pm - 2pm – Fastest / Tue, Nov 24 – ...",£799.00,Wed 25 Nov – Free,...,"85,800円",水 2020/11/25 – 送料無料,"876,30 €",Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,79 990.00 pyб.,Ср 25 Ноя – Бесплатно,星期三 2020/11/25 – 免费,水 2020/11/25 – 送料無料,Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,Ср 25 Ноя – Бесплатно
2,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно,$829.00,"Tomorrow 12pm - 2pm – Fastest / Tue, Nov 24 – ...",£799.00,Wed 25 Nov – Free,...,"85,800円",水 2020/11/25 – 送料無料,"876,30 €",Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,79 990.00 pyб.,Ср 25 Ноя – Бесплатно,星期三 2020/11/25 – 免费,水 2020/11/25 – 送料無料,Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,Ср 25 Ноя – Бесплатно
3,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|2020/12/03 - 2020/12/08 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно,$829.00,"Tomorrow 12pm - 2pm – Fastest / Tue, Nov 24 – ...",£799.00,Wed 25 Nov – Free,...,"85,800円",水 2020/11/25 – 送料無料,"876,30 €",Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,79 990.00 pyб.,Ср 25 Ноя – Бесплатно,2020/12/03 - 2020/12/08 – 免费,水 2020/11/25 – 送料無料,Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,Ср 25 Ноя – Бесплатно
4,$34.54/mo. or $829.00|Tomorrow 12pm - 2pm – Fa...,£33.29/mo. or £799.00**|Wed 25 Nov – Free,RMB 262/月或 RMB 6299|星期三 2020/11/25 – 免费,"85,800円（税別）|水 2020/11/25 – 送料無料","36,51 € /Monat oder 876,30 €**|Heute 19 Uhr - ...",79 990.00 pyб.|Ср 25 Ноя – Бесплатно,$829.00,"Tomorrow 12pm - 2pm – Fastest / Tue, Nov 24 – ...",£799.00,Wed 25 Nov – Free,...,"85,800円",水 2020/11/25 – 送料無料,"876,30 €",Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,79 990.00 pyб.,Ср 25 Ноя – Бесплатно,星期三 2020/11/25 – 免费,水 2020/11/25 – 送料無料,Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,Ср 25 Ноя – Бесплатно
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,$18.70/mo. or $449.00|Tomorrow 12pm - 2pm – Fa...,£18.70/mo. or £449.00**|Wed 25 Nov – Free,RMB 158/月或 RMB 3799|明天 – 免费,"49,800円（税別）|木 2020/11/26 – 送料無料","21,48 € /Monat oder 515,65 €**|Heute 19 Uhr - ...",44 990.00 pyб.|Чт 26 Ноя – Бесплатно,$449.00,"Tomorrow 12pm - 2pm – Fastest / Tue, Nov 24 – ...",£449.00,Wed 25 Nov – Free,...,"49,800円",木 2020/11/26 – 送料無料,"515,65 €",Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,44 990.00 pyб.,Чт 26 Ноя – Бесплатно,明天 – 免费,木 2020/11/26 – 送料無料,Heute 19 Uhr - 22 Uhr – Schnellste / Morgen – ...,Чт 26 Ноя – Бесплатно
59,$18.70/mo. or $449.00|Tomorrow 12pm - 2pm – Fa...,£18.70/mo. or £449.00**|Wed 25 Nov – Free,RMB 158/月或 RMB 3799|明天 – 免费,"49,800円（税別）|木 2020/11/26 – 送料無料","21,48 € /Monat oder 515,65 €*

In [107]:
df_b.to_csv("translated_11.23")